# Resampling Methods - Lab

## Introduction

Now that you have some preliminary background on bootstrapping, jackknife, and permutation tests, its time to practice those skills by coding them into functions. You'll then apply these tests to a hypothesis test and compare the results to a parametric t-test.

## Objectives

In this lab you will: 

* Create functions that perform resampling techniques and use them on datasets

## Bootstrap sampling


Bootstrap sampling works by combining two distinct samples into a universal set and generating random samples from this combined sample space in order to compare these random splits to the two original samples. The idea is to see if the difference between the two **original** samples is statistically significant. If similar differences can be observed through the random generation of samples, then the observed differences are not actually significant.


Write a function to perform bootstrap sampling. The function should take in two samples A and B. The two samples need not be the same size. From this, create a universal sample by combining A and B. Then, create a resampled universal sample of the same size using random sampling with replacement. Finally, split this randomly generated universal set into two samples which are the same size as the original samples, A and B. The function should return these resampled samples.

Example:

```python

A = [1,2,3]
B = [2,2,5,6]

Universal_Set = [1,2,2,2,3,5,6]
Resampled_Universal_Set = [6, 2, 3, 2, 1, 1, 2] # Could be different (randomly generated with replacement)

Resampled_A = [6,2,3]
Resampled_B = [2,1,1,2]
```

In [1]:
import numpy as np
import pandas as pd
import random

In [28]:
def bootstrap(A, B):
    combined = A + B
    resampled = np.random.choice(combined, size=len(combined), replace=True)
    resampled_A = resampled[:len(A)]
    resampled_B = resampled[len(A):]
    
    return resampled_A, resampled_B

In [29]:
A = [1,2,3]
B = [2,2,5,6]
r_A, r_B = bootstrap(A, B)
print(r_A, r_B)

[6 1 2] [1 3 5 2]


## Jackknife 

Write a function that creates additional samples by removing one element at a time. The function should do this for each of the `n` items in the original sample, returning `n` samples, each with `n-1` members.

In [4]:
def jack1(data):
    """This function should take in a list of n observations and return n lists
    each with one member (presumably the nth) removed."""
    n = len(data)
    samples = []
    for i in range(n):
        new_list = data.copy()
        new_list.pop(i)
        samples.append(new_list)
    
    return samples

In [5]:
test = [1,2,2,3,8,9]
jack1(test)

[[2, 2, 3, 8, 9],
 [1, 2, 3, 8, 9],
 [1, 2, 3, 8, 9],
 [1, 2, 2, 8, 9],
 [1, 2, 2, 3, 9],
 [1, 2, 2, 3, 8]]

## Permutation testing

Define a function that generates all possible, equally sized, two set splits of two sets A and B. Sets A and B need not be the same size, but all of the generated two set splits should be of equal size. For example, if we had a set with 5 members and a set with 7 members, the function would return all possible 5-7 ordered splits of the 12 items.

> Note that these are actually combinations! However, as noted previously, permutation tests really investigate possible regroupings of the data observations, so calculating combinations is a more efficient approach!


Here's a more in depth example:

```python
A = [1, 2, 2]
B = [1, 3]
combT(A, B) 
[([1,2,2], [1,3]),
 ([1,2,3], [1,2]),
 ([1,2,1], [2,3]),
 ([1,1,3], [2,2]),
 ([2,2,3], [1,1])]
               
```  

These are all the possible 3-2 member splits of the 5 elements: 1, 1, 2, 2, 3. 

In [6]:
import itertools

In [7]:
def combT(a,b):
    universal_set = sorted(a + b)
    combinations = set(itertools.combinations(universal_set, len(a)))
    groupings = []
    for combination in combinations:
        temp_list = universal_set.copy()
        for element in combination:
            temp_list.remove(element)
        groupings.append((list(combination), temp_list))
    
    return groupings

In [8]:
A = [1, 2, 2]
B = [1, 3]
combT(A,B)

[([1, 1, 3], [2, 2]),
 ([1, 2, 2], [1, 3]),
 ([1, 2, 3], [1, 2]),
 ([1, 1, 2], [2, 3]),
 ([2, 2, 3], [1, 1])]

## Permutation testing in Practice
Let's further investigate the scenario proposed in the previous lesson. Below are two samples A and B. The samples are mock data for the blood pressure of sample patients. The research study is looking to validate whether there is a statistical difference in the blood pressure of these two groups using a 5% significance level.  First, calculate the mean blood pressure of each of the two samples. Then, calculate the difference of these means. From there, use your `combT()` function, defined above, to generate all the possible combinations of the entire sample data into A-B splits of equivalent sizes as the original sets. For each of these combinations, calculate the mean blood pressure of the two groups and record the difference between these sample means. The full collection of the difference in means between these generated samples will serve as the denominator to calculate the p-value associated with the difference between the original sample means.

For example, in our small handwritten example above:

$\mu_a = \frac{1+2+2}{3} = \frac{5}{3}$  
and  
$\mu_b = \frac{1+3}{2} = \frac{4}{2} = 2$  

Giving us

$\mu_a - \mu_b = \frac{5}{3} - 2 = \frac{1}{2}$

In comparison, for our various combinations we have:

([1,2,2], [1,3]):  $\mu_a - \mu_b = \frac{5}{3} - 2 = \frac{1}{2}$  
([1,2,3], [1,2]):  $\mu_a - \mu_b = 2 - \frac{3}{2} = \frac{1}{2}$  
([1,2,1], [2,3]):  $\mu_a - \mu_b = \frac{4}{3} - \frac{5}{3} = -\frac{1}{2}$  
([1,1,3], [2,2]):  $\mu_a - \mu_b = \frac{5}{3} - 2 = \frac{1}{2}$  
([2,2,3], [1,1]):  $\mu_a - \mu_b = \frac{7}{3} - 1 = \frac{4}{3}$  

A standard hypothesis test for this scenario might be:

$H_0: \mu_a = \mu_b$  
$H_1: \mu_a < \mu_b$  
  
Thus comparing our sample difference to the differences of our possible combinations, we look at the number of experiments from our combinations space that were the same or greater than our sample statistic, divided by the total number of combinations. In this case, 4 out of 5 of the combination cases produced the same or greater differences in the two sample means. This value .8 is a strong indication that we cannot refute the null hypothesis for this instance.

In [9]:
a = [109.6927759 , 120.27296943, 103.54012038, 114.16555857,
       122.93336175, 110.9271756 , 114.77443758, 116.34159338,
       112.66413025, 118.30562665, 132.31196515, 117.99000948]
b = [123.98967482, 141.11969004, 117.00293412, 121.6419775 ,
       123.2703033 , 123.76944385, 105.95249634, 114.87114479,
       130.6878082 , 140.60768727, 121.95433026, 123.11996767,
       129.93260914, 121.01049611]

In [10]:
# Your code here
groupings = combT(a, b)
# ⏰ Expect your code to take several minutes to run

In [17]:
# Calculating the difference in sample means:
diff = np.abs(np.mean(a)-np.mean(b))
print("Difference between original samples:", diff)

Difference between original samples: 8.049348947857169


In [18]:
differences = []
for grouping in groupings:
    mean1 = np.mean(grouping[0])
    mean2 = np.mean(grouping[1])
    difference = np.abs(mean1-mean2)
    differences.append(difference)
    
difference_array = np.array(differences)

In [21]:
print("Number of combinations:", len(differences))

Number of combinations: 9657700


In [19]:
p_val = sum(difference_array > diff) / len(differences)
print("p-value from Permutation test:", p_val)

p-value from Permutation test: 0.022695465794133177


In [ ]:
# The p-value is < 0.05, so we can reject the null hypothesis with 95% confidence, and say that
# the two samples are taken from different populations.

## T-test revisited

The parametric statistical test equivalent to our permutation test above would be a t-test of the two groups. Perform a t-test on the same data above in order to calculate the p-value. How does this compare to the above results?

In [22]:
import scipy.stats as stats

In [23]:
# Your code here
statistic, p_val = stats.ttest_ind(a, b, equal_var=False)
print(statistic, p_val)

-2.475967750872386 0.020794009741792126


In [24]:
# The result from the 2-sample t-test is very similar to that above, with a p-val of ~2%, and
# thus a rejection of the null hypothesis that the two samples are from identical populations

## Bootstrap applied

Use your code above to apply the bootstrap technique to this hypothesis testing scenario. Here's a pseudo-code outline for how to do this:

1. Compute the difference between the sample means of A and B
2. Initialize a counter for the number of times the difference of the means of resampled samples is greater then or equal to the difference of the means of the original samples
3. Repeat the following process 10,000 times:
    1. Use the bootstrap sampling function you used above to create new resampled versions of A and B 
    2. Compute the difference between the means of these resampled samples 
    3. If the difference between the means of the resampled samples is greater then or equal to the original difference, add 1 the counter you created in step 2
4. Compute the ratio between the counter and the number of simulations (10,000) that you performed
    > This ratio is the percentage of simulations in which the difference of sample means was greater than the original difference

In [30]:
# Your code here
bigger_diffs = 0
num_trials = 10000

for i in range(num_trials):
    boot_A, boot_B = bootstrap(a,b)
    mean_A = np.mean(boot_A)
    mean_B = np.mean(boot_B)
    trial_diff = np.abs(mean_A-mean_B)
    
    if trial_diff >= diff:
        bigger_diffs += 1
        
p_val = bigger_diffs / num_trials
print("p-value calculated through Bootstrapping:", p_val)

p-value calculated through Bootstrapping: 0.0239


In [ ]:
# The Bootstrapping method has produced another similar p-value just over 2%, resulting in
# another case where the null hypothesis can be rejected.

## Summary

Well done! In this lab, you practice coding modern statistical resampling techniques of the 20th century! You also started to compare these non-parametric methods to other parametric methods such as the t-test that we previously discussed.